In [1]:
# !pip install squarify

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import pandas as pd
import numpy as np

In [3]:
def df_RMF_preprocessing(df):
    # drop rows where Quantity < 0
    df = df[df.Quantity >= 0]
    # drop rows where CustomerID == null
    df = df[df.CustomerID.notnull()]
    # drop rows where UnitPrice < 0
    df = df[df.UnitPrice >= 0]
    # drop duplicated rows
    df = df.drop_duplicates()
    # Convert column 'InvoiceDate' to datetime datatype
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    # convert InvoiceNo data type to integer
    df.InvoiceNo = df.InvoiceNo.astype(int)
    # Create new column 'Amount'
    df['Amount'] = df['Quantity'] * df['UnitPrice']
    # Get max date of dataframe
    max_date = df['InvoiceDate'].max().date()
    # Calculate R, F, M
    Recency = lambda x: (max_date - x.max().date()).days
    Frequency = lambda x: x.nunique()
    Monetary = lambda x: round(sum(x),2)

    df_RFM = df.groupby('CustomerID').agg({'InvoiceDate' : Recency,
                                        'InvoiceNo' : Frequency,
                                        'Amount' : Monetary,
                                        })
    # Rename column names
    df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
    df_RFM = df_RFM.sort_values('Monetary', ascending=False)
    return df_RFM

In [4]:
# Calculate average values for each GMM_segment, and return a size of each segment 
def create_df_agg(df, groupby_col):
    df.R = df.R.astype(int)
    df.F = df.F.astype(int)
    df.M = df.M.astype(int)
    
    df_agg = df.groupby(groupby_col).agg({
      'Recency': 'mean',
      'Frequency': 'mean',
      'Monetary': 'mean',
      'R': 'mean',
      'F': 'mean',
      'M': ['mean', 'count']}).round(0)

    df_agg.columns = df_agg.columns.droplevel()
    df_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'RMean','FMean','MMean','Count']
    df_agg['Percent'] = round((df_agg['Count']/df_agg.Count.sum())*100, 2)

    # Reset the index
    df_agg = df_agg.reset_index()
    return df_agg

In [5]:
# Visualization - Treemap
def treemap_customer_segmentation(df_agg,font_size):
    #Create our plot and resize it.
    fig = plt.gcf()
    ax = fig.add_subplot()
    fig.set_size_inches(22, 9)

    # create a color palette, mapped to these values
    cmap = matplotlib.cm.plasma
    norm = matplotlib.colors.Normalize(vmin=min(df_agg['Count']), vmax=max(df_agg['Count']))
    colors = [cmap(norm(value)) for value in df_agg['Count']]

    squarify.plot(sizes=df_agg['Count'],
                text_kwargs={'fontsize':font_size,'weight':'bold', 'fontname':"sans serif"},
                color=colors,
                label=['{} \n{:.0f} days \n{:.0f} orders \n{:.0f} $ \n{:.0f} customers ({}%)'.format(*df_agg.iloc[i])
                        for i in range(0, len(df_agg))], alpha=0.5 )


    plt.title("Customers Segments",fontsize=26,fontweight="bold")
    plt.axis('off')

    # plt.savefig('RFM Segments.png')
    plt.show()